<a href="https://colab.research.google.com/github/Seeed-Studio/Edgelab/blob/master/docs/Analog_meter_detection_own_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Collect, train and deploy an analog meter reading detection model with your own dataset

This notebook will guide you to train an analog meter reading detection model with your own dataset!

## 1. Configure host environment

**Step 1.** Choose **GPU** in **Runtime** if not already selected by navigating to `Runtime --> Change Runtime Type --> Hardware accelerator --> GPU`

**Step 2.** Clone Edgelab repo and access it

In [ ]:
%cd ~
!git clone https://github.com/Seeed-Studio/Edgelab
%cd Edgelab

**Step 3.** Configure the environment by running the following script which will download and install the relevant dependencies

In [ ]:
!python tools/env_config.py

**Step 4.** After the relevant environment configuration is completed, the path of this project needs to be configured in the PYTHONPATH environment variable. Configure it as follows

In [ ]:
import os
home = os.environ['HOME']
os.environ['PYTHONPATH'] = f'{home}/Edgelab'

## 2. Generate firmware image

We need to generate a firmware image for the Grove - Vision AI to support the analog meter reading detection model because the default firmware which is pre-installed out-of-the-box does not support it.

If you want to download an already pre-compiled firmware, please follow the links below. But we recommend you to compile the firmware from source, so that you will have the latest firmware always.

- [Grove - Vision AI Module](https://files.seeedstudio.com/wiki/Edgelab/uf2/grove-vision-ai-firmware.uf2)
- [SenseCAP A1101](https://files.seeedstudio.com/wiki/Edgelab/uf2/sensecap-A1101-firmware.uf2)

**Step 1.** Download GNU Development Toolkit

In [ ]:
%cd ~
!wget https://github.com/foss-for-synopsys-dwc-arc-processors/toolchain/releases/download/arc-2020.09-release/arc_gnu_2020.09_prebuilt_elf32_le_linux_install.tar.gz

**Step 2.** Extract the file

In [ ]:
!tar -xvf arc_gnu_2020.09_prebuilt_elf32_le_linux_install.tar.gz

**Step 3:** Add **arc_gnu_2020.09_prebuilt_elf32_le_linux_install/bin** to **PATH**

In [ ]:
os.environ['PATH'] += f':{home}/arc_gnu_2020.09_prebuilt_elf32_le_linux_install/bin'

**Step 4:** Execute the following to install required dependencies 

In [ ]:
%cd
!wget -4c https://ftp.gnu.org/gnu/glibc/glibc-2.29.tar.gz
!tar -zxvf glibc-2.29.tar.gz
!apt-get install gawk bison -y
%cd glibc-2.29 
%mkdir build
%cd build
!../configure --prefix=/usr/local --disable-sanity-checks
!make -j8
!make install
%cd /lib/x86_64-linux-gnu
!cp /usr/local/lib/libm-2.29.so /lib/x86_64-linux-gnu/
!ln -sf libm-2.29.so libm.so.6

**Step 5:** Navigate to the following repo of Edgelab

In [ ]:
%cd ~/Edgelab/examples/vision_ai

**Step 6:** Download related tools


In [ ]:
!make download

**Step 7:** Compile the firmware according to your hardware 

- For Grove - Vision AI Module

In [ ]:
!make HW=grove_vision_ai APP=meter
!make flash

- For SenseCAP A1101

In [ ]:
!make HW=sensecap_vision_ai APP=meter
!make flash

The above will generate output.img inside **~/Edgelab/examples/vision_ai/tools/image_gen_cstm/output/** directory

**Step *:** Generate firmware image **firmware.uf2** file so that we can later flash it directly to the Grove - Vision AI Module/ SenseCAP A1101

In [ ]:
!python3 tools/ufconv/uf2conv.py -t 0 -c tools/image_gen_cstm/output/output.img -o firmware.uf2

This will generate **firmware.uf2** inside **~/Edgelab/examples/vision_ai** directory



**Step 8:** Download the **firmware.uf2** file

In [ ]:
from google.colab import files
files.download("firmware.uf2")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 3. Prepare dataset

Here we explain how to use your own dataset

### Collect dataset

If you want to train your own analog meter detection model for a specific application, you need to spend sometime to collect images to prepare a dataset. Here you can take several photos (start with 200 and go higher to improve accuracy) of the analog meter that you want to detect with the meter pointer at different points and also take photos at different lighting conditions and different environments as follows

#### Pointer reading at dark environment 

<div align=center><img width=350 src="https://files.seeedstudio.com/wiki/Edgelab/meter-own-github/9.jpg"/></div>

#### Pointer reading at light environment 

<div align=center><img width=350 src="https://files.seeedstudio.com/wiki/Edgelab/meter-own-github/10.jpg"/></div>

### Annotate dataset

Next we need to annotate all the images that we have collected. Here we will use an application called **labelme** which is an open source image annotation tool.

- **Step 1.** Visit [this page](https://github.com/wkentaro/labelme#installation) and install labelme according to your operating system

- **Step 2.** On the command-line, type the following to open **labelme**

```sh
labelme
```

- **Step 3.** Once labelme opens, click on **OpenDir**, select the folder that you have put all the collected images and click **Select Folder**

<div align=center><img width=350 src="https://files.seeedstudio.com/wiki/Edgelab/meter-own-github/1.jpg"/></div>

<div align=center><img width=550 src="https://files.seeedstudio.com/wiki/Edgelab/meter-own-github/2.png"/></div>

- **Step 4.** Later, when we annotate images, labelme will generate a **json file** for each image and this file will contain the annotation information for the corresponsing image. Here we need to specify a directory to store these image annotations because we recommend to store these json files and image files in 2 different folders. Go to `File > Change Output Dir`

<div align=center><img width=250 src="https://files.seeedstudio.com/wiki/Edgelab/meter-own-github/3.jpg"/></div>

- **Step 5.** Create a new folder, select the folder and click **Select Folder** 

<div align=center><img width=550 src="https://files.seeedstudio.com/wiki/Edgelab/meter-own-github/4.jpg"/></div>

- **Step 6.** Go to `File > Save Automatically` to save time when annotating all the images. Otherwise it will pop up a prompt to save each image.

<div align=center><img width=250 src="https://files.seeedstudio.com/wiki/Edgelab/meter-own-github/5.png"/></div>

- **Step 7.** Right click on the first opened image and select **Create Point**

<div align=center><img width=550 src="https://files.seeedstudio.com/wiki/Edgelab/meter-own-github/6.jpg"/></div>

- **Step 8.** Draw a point at the tip of the pointer, set any label name and click **OK**

<div align=center><img width=350 src="https://files.seeedstudio.com/wiki/Edgelab/meter-own-github/25.png"/></div>

<div align=center><img width=550 src="https://files.seeedstudio.com/wiki/Edgelab/meter-own-github/26.png"/></div>

After the point, there will be a new **json file** created automatically for each image file under "annotations" folder as mentioned before.

### Organize dataset

Now you need to manually organize the dataset by splitting all the images and annotations into **train, val, test** directories as follows

Here we recommend you to split in the following percentages

- train = 80%
- val = 10%
- test = 10%

So for example, if you have 200 images, the split would be

- train = 160 images
- val = 20 images
- test = 20 images

```
meter_data
    |train
        |images
            |a.jpg
            |b.jpg
        |annotations
            |a.json
            |b.json
    |val
        |images
            |c.jpg
            |d.jpg
        |annotations
            |c.json
            |d.json
    |test
        |images
            |e.jpg
            |f.jpg
        |annotations
            |e.json
            |f.json
```

## 4. Start training

### Configuration file 

We will choose the profile according to the task that we want to implement. We have prepared preconfigured files inside the the [configs](https://github.com/Seeed-Studio/edgelab/tree/master/configs) folder.

For our meter reading detection example, we will use [pfld_mv2n_112.py](https://github.com/Seeed-Studio/Edgelab/blob/master/configs/pfld/pfld_mv2n_112.py) config file. This file will be mainly used to configure the dataset for training including the dataset location.

Here you can use your own dataset for training.
Execute the following command inside the activated conda virtual environment terminal to start training an end-to-end analog meter reading detection model.

In [ ]:
!~/anaconda3/envs/edgelab/bin/python tools/train.py mmpose configs/pfld/pfld_mv2n_112.py --gpus=1 --cfg-options runner.max_epochs=100 data.train.index_file=/meter/train/annotations data.train.img_dir=/meter/train/images data.val.index_file=/meter/val/annotations data.val.img_dir=/meter/val/images data.test.index_file=/meter/test/annotations data.test.img_dir=/meter/test/images

**Note:** Make sure to change the **img_dir** and **index_file** locations according to your dataset images and annotations path

The format of the above command looks like below

```sh
python tools/train.py <task_type> <config_file_location> --gpus=<cpu_or_gpu> --cfg-options runner.max_epochs=<number_of_epochs> data.train.index_file=<absolute_path_to_annotations_in_train> data.train.img_dir=<absolute_path_to_images_in_train> data.val.index_file=<absolute_path_to_annotations_in_val> data.val.img_dir=<absolute_path_to_images_in_val> data.test.index_file=<absolute_path_to_annotations_in_test> data.test.img_dir=<absolute_path_to_images_in_test>
```

where:

- <task_type> refers to either **mmcls** for classfication, **mmdet** for detection and **mmpose** for pose estimation
- <config_file_location> refers to the path where the model configuration is located 
- <cpu_or_gpu> refers to specifying whether you want to train on CPU or GPU. Type **0** CPU and **1** for GPU
- --cfg-options runner.max_epochs=<number_of_epochs> refers to the number of training cycles
- --cfg-options data.train.index_file=<absolute_path_to_annotations_in_train> refers to the location of the annotations files under train set
- --cfg-options data.train.img_dir=<absolute_path_to_images_in_train> refers to the location of the images files under train set
- --cfg-options data.val.index_file=<absolute_path_to_annotations_in_val> refers to the location of the annotations files under validation set
- --cfg-options data.val.index_file=<absolute_path_to_annotations_in_val> refers to the location of the image files under validation set
- --cfg-options data.test.index_file=<absolute_path_to_annotations_in_test> refers to the location of the annotations files under test set
- --cfg-options data.test.img_dir=<absolute_path_to_images_in_test> refers to the location of the image files under test set

## 5. Export PyTorch to TFLite

After the model training is completed, you can export the **.pth** file to the **TFLite** file format. This is important because TFLite format is more optimized to run on low power hardware. Assuming that the environment is in this project path, you can export the models you have trained before to the TFLite format by running the following command:



In [ ]:
!python tools/export.py configs/pfld/pfld_mv2n_112.py --weights work_dirs/pfld_mv2n_112/exp1/latest.pth --data ~/datasets/meter/train/images

This will generate a **latest_int8.tflite** file inside **~/Edgelab/work_dirs/pfld_mv2n_112/exp1** directory

The format of the above command looks like below

```sh
python tools/export.py configs/xxx/xxx.py --weights <location_to_pth_from_training> --data <location_to_images_directory_of__train_or_val>
```

where:

- configs/xxx/xxx.py refers to the location of the configuration file correcsponsing to the AI model
- --weights refers to the the .pth file that was generated during training
- --data refers to the images directory of either train or val

## 6. Convert TFLite to UF2

Now we will convert the generated TFLite file to a UF2 file so that we can directly flash the UF2 file into Grove - Vision AI Module and SenseCAP A1101



**Step 1:** Execute the following

In [ ]:
!python Edgelab/examples/vision_ai/tools/ufconv/uf2conv.py -f GROVEAI -t 1 -c ~/Edgelab/work_dirs/pfld_mv2n_112/exp1/latest_int8.tflite -o model.uf2

This will generate a **model.uf2** file inside **~/Edgelab/examples/vision_ai** directory



Here you only change the location of the TFLite model such as **~/Edgelab/work_dirs/pfld_mv2n_112/exp1/latest_int8.tflite**



**Step 2:** Download the uf2 model file

In [ ]:
files.download("model.uf2")

## 7. Deploy and inference

### Flash firmware and model

This explains how you can flash the previously generated firmware (firmware.uf2) and the model file (model.uf2) to Grove - Vision AI Module and SenseCAP A1101.

**Step 1.** Connect Grove - Vision AI Module/ SenseCAP A1101 to PC by using USB Type-C cable 

<div align=center><img width=1000 src="https://files.seeedstudio.com/wiki/SenseCAP-A1101/45.png"/></div>

**Step 2.** Double click the boot button to enter **boot mode**

<div align=center><img width=1000 src="https://files.seeedstudio.com/wiki/SenseCAP-A1101/46.png"/></div>

**Step 3:** After this you will see a new storage drive shown on your file explorer as **GROVEAI** for **Grove - Vision AI Module** and as **VISIONAI** for **SenseCAP A1101**

<div align=center><img width=500 src="https://files.seeedstudio.com/wiki/SenseCAP-A1101/62.jpg"/></div>

**Step 4:** Drag and drop the previous **firmware.uf2** at first, and then the **model.uf2** file to **GROVEAI** or **VISIONAI** 

Once the copying is finished **GROVEAI** or **VISIONAI** drive will disapper. This is how we can check whether the copying is successful or not.

## 8. View live detection results

**Step 1:** After loading the firmware and connecting to PC, visit [this URL](https://files.seeedstudio.com/grove_ai_vision/index.html)

**Step 2:** Click **Connect** button. Then you will see a pop up on the browser. Select **Grove AI - Paired** and click **Connect**
  
<div align=center><img width=800 src="https://files.seeedstudio.com/wiki/Edgelab/meter-own-github/13.jpg"/></div>

<div align=center><img width=400 src="https://files.seeedstudio.com/wiki/Edgelab/meter-own-github/12.png"/></div>

Upon successful connection, you will see a live preview from the camera. Here the camera is pointed at an analog meter.

<div align=center><img width=800 src="https://files.seeedstudio.com/wiki/Edgelab/meter-own-github/14.png"/></div>

Now we need to set 3 points which is the center point, start point and the end point. 

**Step 3:** Click on **Set Center Point** and click on the center of the meter. you will see a pop up confirm the location and press **OK**

<div align=center><img width=800 src="https://files.seeedstudio.com/wiki/Edgelab/meter-own-github/15.png"/></div>

You will see the center point is already recorded

<div align=center><img width=800 src="https://files.seeedstudio.com/wiki/Edgelab/meter-own-github/16.png"/></div>

**Step 4:** Click on **Set Start Point** and click on the first indicator point. you will see a pop up confirm the location and press **OK**

<div align=center><img width=800 src="https://files.seeedstudio.com/wiki/Edgelab/meter-own-github/17.png"/></div>

You will see the first indicator point is already recorded

<div align=center><img width=800 src="https://files.seeedstudio.com/wiki/Edgelab/meter-own-github/18.png"/></div>

**Step 5:** Click on **Set End Point** and click on the last indicator point. you will see a pop up confirm the location and press **OK**

<div align=center><img width=800 src="https://files.seeedstudio.com/wiki/Edgelab/meter-own-github/19.png"/></div>

You will see the last indicator point is already recorded

<div align=center><img width=800 src="https://files.seeedstudio.com/wiki/Edgelab/meter-own-github/20.png"/></div>

**Step 6:** Set the measuring range according to the first digit and last digit of the meter. For example, he we set as **From:0 To 0.16**

<div align=center><img width=800 src="https://files.seeedstudio.com/wiki/Edgelab/meter-own-github/21.png"/></div>

**Step 7:** Set the number of decimal places that you want the result to display. Here we set as 2

<div align=center><img width=800 src="https://files.seeedstudio.com/wiki/Edgelab/meter-own-github/22.png"/></div>

Finally you can see the live meter reading results as follows

<div align=center><img width=800 src="https://files.seeedstudio.com/wiki/Edgelab/meter-own-github/meter.gif"/></div>